In [142]:
import csv
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import numpy as np
import pandas as pd

In [143]:
categories = ['airlines', 'celebrities', 'colleges', 'fastfood', 
              'leagues', 'news', 'streamingplatforms', 'techgiants']

subCategories = [['output_AmericanAir', 'output_JetBlue', 'output_qatarairways', 'output_SingaporeAir'], 
                 ['output_TheRock', 'output_ArianaGrande', 'output_chrissyteigen', 'output_jimmyfallon'], 
                 ['output_Stanford', 'output_Harvard', 'output_MIT', 'output_UCBerkeley'], 
                 ['output_McDonalds', 'output_BurgerKing', 'output_Starbucks', 'output_wendys'], 
                 ['output_IPL', 'output_MLB', 'output_MLS', 'output_NBA'], 
                 ['output_BBCWorld', 'output_CNN', 'output_washingtonpost', 'output_nytimes'], 
                 ['output_Netflix', 'output_Spotify', 'output_PrimeVideo', 'output_HBO'], 
                 ['output_amazon', 'output_facebook', 'output_Tesla', 'output_Microsoft']]

In [144]:
sizeOfSets = 400

categCount = 0
for i in range(len(categories)):
    for j in subCategories[i]:
        categCount += 1
        
categCount *= sizeOfSets

val = [None] * categCount # Keep value (sizeOfSets * no. of categories)
sizeOfTweetContent = 75

print(len(val)) # No. Of Posts

12800


In [145]:
def inputDataInDataframe():
    overCount= 0
    for i in range(len(categories)):
        for j in subCategories[i]:
            direc:str = '/Users/sameernawaz/AnacondaProjects/Datasets/tweets/'+categories[i]+'/'+j+'.csv'
            with open(direc, 'r') as csv_file:
                csv_reader = csv.reader(csv_file)
                count = 0
                for line in csv_reader:
                    if len(line[6]) > sizeOfTweetContent:
                        val[overCount] = line
                        count += 1
                        overCount += 1
                        if count >= sizeOfSets :
                            break

In [146]:
inputDataInDataframe()

In [147]:
df = pd.DataFrame(val, columns = ['date', 'username', 'to', 'replies', 
                                  'retweets', 'favorites', 'text', 'geo', 
                                  'mentions', 'hashtags', 'id', 'permalink']) 

In [148]:
# Once data added in dataframe, Shuffle it
df = df.sample(frac=1).reset_index(drop=True)
# Add a unique id (rec_id) for all tweets
df = df.assign(rec_id=np.arange(len(df))).reset_index(drop=True)

In [149]:
df = df[['rec_id', 'username', 'text']]
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12800 entries, 0 to 12799
Data columns (total 3 columns):
rec_id      12800 non-null int64
username    12800 non-null object
text        12800 non-null object
dtypes: int64(1), object(2)
memory usage: 400.0+ KB


In [150]:
import nltk
import re
import numpy as np
import contractions

stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    doc = contractions.fix(doc)
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    #filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

norm_corpus = normalize_corpus(list(df['text']))
len(norm_corpus)

12800

In [151]:
# Extract TF-IDF

from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(ngram_range=(1, 2), min_df=2)
tfidf_matrix = tf.fit_transform(norm_corpus)
tfidf_matrix.shape

(12800, 26984)

In [152]:
# Compute Pairwise Document Similarity

from sklearn.metrics.pairwise import cosine_similarity

doc_sim = cosine_similarity(tfidf_matrix)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

,0,1,2,3,4,5,6,7,8,9,...,12790,12791,12792,12793,12794,12795,12796,12797,12798,12799
0,1.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.045305,0.0,0.0,0.0,0.0,0.0
1,0.0,1.000000,0.000000,0.054008,0.000000,0.0,0.0,0.0,0.0,0.027313,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
2,0.0,0.000000,1.000000,0.000000,0.013809,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
3,0.0,0.054008,0.000000,1.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0
4,0.0,0.000000,0.013809,0.000000,1.000000,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0


In [153]:
# Get list of post IDs

posts_list = df['rec_id'].values
posts_list, posts_list.shape

(array([    0,     1,     2, ..., 12797, 12798, 12799]), (12800,))

In [154]:
posts_idx = np.where(posts_list == 3997)[0][0]
posts_idx

3997

In [155]:
posts_similarities = doc_sim_df.iloc[posts_idx].values
posts_similarities

array([0., 0., 0., ..., 0., 0., 0.])

In [156]:
# Get top 5 similar post IDs

similar_posts_idxs = np.argsort(-posts_similarities)[1:6]
similar_posts_idxs

array([ 4535,  8247,  7841,  4486, 11595])

In [157]:
similar_posts = posts_list[similar_posts_idxs]
similar_posts

array([ 4535,  8247,  7841,  4486, 11595])

In [158]:
def relevant_posts(postId, posts = posts_list, doc_sims = doc_sim_df):
    # find post id
    posts_idx = np.where(posts == postId)[0][0]
    # get posts similarities
    posts_similarities = doc_sims.iloc[posts_idx].values
    # get top 5 similar post IDs
    similar_posts_idxs = np.argsort(-posts_similarities)[1:6]
    # get top 5 posts
    similar_posts = posts[similar_posts_idxs]
    # return the top 5 posts
    return similar_posts

In [165]:
post_id = 10059

print('-----Input Post-----')
print((df.loc[df['rec_id'] == post_id]['text']).values[0])
print('\n-----Relevant Posts-----')
for i in relevant_posts(postId = post_id, posts = posts_list, doc_sims = doc_sim_df):
    print("=>",(df.loc[df['rec_id'] == i]['text']).values[0])

-----Input Post-----
More #SeptemberBaseball on @Youtube! Watch @RaysBaseball vs. the @Dodgers tomorrow at 10:00pm ET. Set a reminder for a game time notification https://bit.ly/2kBd2Zn

-----Relevant Posts-----
=> A game with Postseason implications! Watch @Indians-@Angles tomorrow at 10:00pm ET LIVE on @YouTube. Don't forget to set a YouTube reminder to get notified at game time http://bit.ly/2k3oDzQ
=> An NL East showdown between postseason contenders. Watch @Braves-@Nationals tomorrow at 7 pm ET on @YouTube. Set a reminder for game time: https://atmlb.com/31fvsi8
=> #SeptemberBaseball on @YouTube! It’s @Mets - @Nationals tomorrow at 1:00pm. Don't forget to set a reminder to get notified at game time. https://bit.ly/2k1Lk7I
=>  FREE @RaysBaseball are taking on the @Dodgers LIVE on @YouTube today at 10:00pm ET. Catch the action https://atmlb.com/2ZZvcXn
=> We have more free baseball tomorrow on @YouTube! Catch the @BlueJays - @RaysBaseball live at 7:00pm ET. Don't forget to set a rem